In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import torchvision as tv
import pandas as pd
from tqdm import tqdm

from models.vgg_16 import BinaryVgg16
from trainer.binary_trainer import BinaryTrainer
from datasets.image_sharpness_ds import ImageSharpnessDS

C:\Users\silva\anaconda3\envs\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dev = th.device("cuda") if th.cuda.is_available() else th.device("cpu")
loaded_net = th.load('C:/workspace/Binary_sharpness/saved_models/model_final.pt')
net2 = BinaryVgg16(None).to(dev)
net2.load_state_dict(loaded_net['model_state_dict'])
net2.eval()

ds = ImageSharpnessDS('C:/workspace/datasets/binary_sharpness/labels_1.csv',
                      'C:/workspace/datasets/binary_sharpness/images',
                      transform=th.nn.Sequential(tv.transforms.Resize((128, 128))
                                                 ))

totalloader = th.utils.data.DataLoader(ds, batch_size=100, shuffle=True, num_workers=0)

loop = tqdm(totalloader)

result = 0

for j, data in enumerate(loop, 0):
    img, target = data
    target = target.to(dev)
    img = img.to(dev)

    pred = net2.forward(img)
    pred = th.where(pred > 0.5, 1.0, 0.0)

    result += th.sum(nn.functional.l1_loss(pred, target, reduction='none')).item()
    print(result)

print(result)



[Modify pytorch standard model](https://glassboxmedicine.com/2020/12/08/using-predefined-and-pretrained-cnns-in-pytorch-tutorial-with-code/)

In [23]:
dev = th.device("cuda") if th.cuda.is_available() else th.device("cpu")

#net = UNetResInc(params).to(dev)
net = BinaryVgg16(None).to(dev)
optim = th.optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)
#optim = th.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.MSELoss()

ds = ImageSharpnessDS('C:/workspace/datasets/binary_sharpness/labels_1.csv',
                      'C:/workspace/datasets/binary_sharpness/images',
                      transform=th.nn.Sequential(tv.transforms.Resize((128, 128
                                                                       ))
                                                 ))

# count = pd.read_csv('C:/workspace/datasets/binary_sharpness/labels_1.csv')
# counter = (len(count['label']) - np.sum(count['label'])) / len(count['label'])

new_ds = []
counter = 0

for i in range(len(ds)):

    label = ds.img_labels.iloc[i, 1]
    if label < 0.5:
        img, label = ds[i]
        new_ds.append((img, label))
        counter += 1

print(counter)
idxs = []

while counter > 0:
    #print(counter)
    randint = np.random.randint(0, len(ds))
    label = ds.img_labels.iloc[randint, 1]
    if label > 0.5 and randint not in idxs:
        new_ds.append(ds[randint])
        counter -= 1
        idxs.append(randint)

# ds, _ = th.utils.data.random_split(ds, [len_keep, len_discard])

ds = new_ds

len_train = int(0.8 * len(ds))
len_test = len(ds) - len_train

ds_train, ds_test = th.utils.data.random_split(ds, [len_train, len_test])

trainloader = th.utils.data.DataLoader(ds_train, 100, shuffle=True, num_workers=0)
testloader = th.utils.data.DataLoader(ds_test, 25, shuffle=True, num_workers=0)

trainer = BinaryTrainer(net, dev, criterion, optim)

for i in range(100):
    img, tar, pre = trainer.train_epoch(trainloader)
    img, tar, pre = trainer.test(testloader)


KeyboardInterrupt: 

In [ ]:
th.save(net.state_dict(), 'C:/workspace/Binary_sharpness/saved_models/model_1.pt')

In [ ]:
for i in range(100):
    img, tar, pre = trainer.train_epoch(trainloader)
    img, tar, pre = trainer.test(testloader)

In [ ]:
th.save({
        'epoch': trainer.epochs,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'loss': trainer.test_losses,
        }, 'C:/workspace/Binary_sharpness/saved_models/model_' + str(trainer.epochs) + '.pt')

In [ ]:
for i in range(100):
    img, tar, pre = trainer.train_epoch(trainloader)
    img, tar, pre = trainer.test(testloader)

In [ ]:
th.save({
        'ds_train': ds_train,
        'ds_test': ds_test,
        'epoch': trainer.epochs,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'trainer': trainer,
        'loss': trainer.test_losses,
        }, 'C:/workspace/Binary_sharpness/saved_models/model_' + str(trainer.epochs) + '.pt')

In [ ]:
for i in range(100):
    img, tar, pre = trainer.train_epoch(trainloader)
    img, tar, pre = trainer.test(testloader)

In [ ]:
th.save({
        'epoch': trainer.epochs,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'loss': trainer.test_losses,
        }, 'C:/workspace/Binary_sharpness/saved_models/model_' + str(trainer.epochs) + '.pt')

In [ ]:
for i in range(100):
    img, tar, pre = trainer.train_epoch(trainloader)
    img, tar, pre = trainer.test(testloader)
    if trainer.epochs % 10 == 0:
        th.save({
        'epoch': trainer.epochs,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'loss': trainer.test_losses,
        }, 'C:/workspace/Binary_sharpness/saved_models/model_' + str(trainer.epochs) + '.pt')

In [ ]:
th.save({
        'ds_train': ds_train,
        'ds_test': ds_test,
        'epoch': trainer.epochs,
        'model_state_dict': net.state_dict(),
        'optimizer_state_dict': optim.state_dict(),
        'trainer': trainer,
        'loss': trainer.test_losses,
        }, 'C:/workspace/Binary_sharpness/saved_models/model_final.pt')

In [ ]:
loaded_net = th.load('C:/workspace/Binary_sharpness/saved_models/model_final.pt')

In [ ]:
net2 = BinaryVgg16(None).to(dev)
net2.load_state_dict(loaded_net['model_state_dict'])

In [ ]:
net2.eval()

In [ ]:

loaded_net = th.load('C:/workspace/Binary_sharpness/saved_models/model_final.pt')
net2 = BinaryVgg16(None).to(dev)
net2.load_state_dict(loaded_net['model_state_dict'])
net2.eval()

totalloader = trainloader = th.utils.data.DataLoader(ImageSharpnessDS('C:/workspace/datasets/binary_sharpness/labels_1.csv',
                      'C:/workspace/datasets/binary_sharpness/images',
                      transform=th.nn.Sequential(tv.transforms.Resize((128, 128
                                                                       ))
                                                 )), 100, shuffle=True)

result = 0

for j, data in enumerate(totalloader, 0):
    img, target = data
    target = target.to(dev)
    img = img.to(dev)

    pred = net2.forward(img)

    loss = th.sum(th.abs(target - pred))

    result += loss

print(result)
